In [1]:
# FiftyOne Method
!pip install --upgrade fiftyone

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz

In [6]:
dataset = foz.load_zoo_dataset(
    "coco-2017",
split = "validation",
label_types = ["detections"],
classes = ["person", "car", "bus", "motorcycle", "bicycle", "truck"],
max_samples = 3000)

Found annotations at 'C:\Users\12108\fiftyone\coco-2017\raw\instances_val2017.json'
Only found 2968 (<3000) samples matching your requirements
1206 images found; downloading the remaining 1762
 100% |████████████████| 1762/1762 [1.5m elapsed, 0s remaining, 12.6 images/s]      
Writing annotations for 2968 downloaded samples to 'C:\Users\12108\fiftyone\coco-2017\validation\labels.json'
Dataset info written to 'C:\Users\12108\fiftyone\coco-2017\info.json'
Loading 'coco-2017' split 'validation'
 100% |███████████████| 2968/2968 [15.7s elapsed, 0s remaining, 216.2 samples/s]      
Dataset 'coco-2017-validation-3000' created


In [4]:
len(dataset)

20000

In [9]:
# # Export filtered dataset
# export_dir = r"C:\Users\12108\SpyderCodes\GradProj"

# dataset.export(
#     export_dir=export_dir,
#     dataset_type=fo.types.COCODetectionDataset,
#     label_field="ground_truth",
#      labels_path=r"C:\Users\12108\SpyderCodes\GradProj\FiftyOneLabels.json",
# )

In [3]:
import json
import os

def coco_to_yolo(annotations, image_width, image_height):
    yolo_annotations = []
    for annotation in annotations:
        category_id = annotation['category_id']
        bbox = annotation['bbox']
        
        # Filter out annotations for specified classes
        if category_id in [1, 2, 3, 4, 6, 7]:  # person, bicycle, car, motorcycle, bus, train
            x_center = bbox[0] + bbox[2] / 2
            y_center = bbox[1] + bbox[3] / 2
            width = bbox[2]
            height = bbox[3]

            # Normalize coordinates
            x_center /= image_width
            y_center /= image_height
            width /= image_width
            height /= image_height

            yolo_annotations.append((category_id, x_center, y_center, width, height))

    return yolo_annotations

input_path = r"C:\Users\12108\fiftyone\raw\instances_val2017.json"
output_path = r"C:\Users\12108\fiftyone\detect\labels\labels_val3"       
image_folder = r"C:\Users\12108\fiftyone\detect\images\val"
                
with open(input_path, 'r') as file:
    data = json.load(file)

# Create YOLO annotations for each image
annotated_images = set()
for annotation in data['annotations']:
    annotated_images.add(annotation['image_id'])

for image_data in data['images']:
    image_id = image_data['id']
    
    # Check if image file exists
    image_path = os.path.join(image_folder, f"{image_id:012d}.jpg")
    if not os.path.exists(image_path):
        print(f"Image {image_id} not found, skipping...")
        continue
    
    # Check if the image has annotations
    if image_id not in annotated_images:
        print(f"No annotations found for image {image_id}, skipping...")
        continue
    
    image_annotations = [annotation for annotation in data['annotations'] if annotation['image_id'] == image_id]
    
    # Get image dimensions
    image_width = image_data['width']
    image_height = image_data['height']
    
    # Convert COCO annotations to YOLO format
    yolo_annotations = coco_to_yolo(image_annotations, image_width, image_height)
    
    if not yolo_annotations:
        print(f"No annotations found for image {image_id}")
        continue
    
    # Format image ID with leading zeros
    image_name = f"{image_id:012d}.txt"
    
    # Writing to YOLO format
    output_file = os.path.join(output_path, image_name)
    with open(output_file, 'w') as f:
        for annotation in yolo_annotations:
            category_id, x_center, y_center, width, height = annotation
            line = f"{category_id} {x_center} {y_center} {width} {height}\n"
            f.write(line)
    
    print(f"Annotations for image {image_id} converted to YOLO format and saved to {output_file}")


Image 397133 not found, skipping...
Image 37777 not found, skipping...
Image 252219 not found, skipping...
Image 87038 not found, skipping...
Image 174482 not found, skipping...
Image 403385 not found, skipping...
Image 6818 not found, skipping...
Image 480985 not found, skipping...
Image 458054 not found, skipping...
Image 331352 not found, skipping...
Image 296649 not found, skipping...
Image 386912 not found, skipping...
Image 502136 not found, skipping...
Image 491497 not found, skipping...
Image 184791 not found, skipping...
Image 348881 not found, skipping...
Image 289393 not found, skipping...
Image 522713 not found, skipping...
Image 181666 not found, skipping...
Image 17627 not found, skipping...
Image 143931 not found, skipping...
Image 303818 not found, skipping...
Image 463730 not found, skipping...
Image 460347 not found, skipping...
Image 322864 not found, skipping...
Image 226111 not found, skipping...
Image 153299 not found, skipping...
Image 308394 not found, skipping.

In [ ]:
import json
import os

def coco_to_yolo(annotations, image_width, image_height):
    yolo_annotations = []
    class_mapping = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}  # Mapping COCO class IDs to YOLO class IDs
    for annotation in annotations:
        category_id = annotation['category_id']
        
        # Map COCO category ID to YOLO category ID
        if category_id not in class_mapping:
            continue
        category_id_yolo = class_mapping[category_id]
        
        bbox = annotation['bbox']
        
        x_center = bbox[0] + bbox[2] / 2
        y_center = bbox[1] + bbox[3] / 2
        width = bbox[2]
        height = bbox[3]

        # Normalize coordinates
        x_center /= image_width
        y_center /= image_height
        width /= image_width
        height /= image_height

        yolo_annotations.append((category_id_yolo, x_center, y_center, width, height))

    return yolo_annotations

input_path = r"C:\Users\12108\fiftyone\raw\instances_train2017.json"
output_path = r"C:\Users\12108\fiftyone\detect\labels\labels_train3"       
image_folder = r"C:\Users\12108\fiftyone\detect\images\train"

with open(input_path, 'r') as file:
    data = json.load(file)

# Create YOLO annotations for each image
for image_data in data['images']:
    image_id = image_data['id']
    
    # Check if image file exists
    image_path = os.path.join(image_folder, f"{image_id:012d}.jpg")
    if not os.path.exists(image_path):
        print(f"Image {image_id} not found, skipping...")
        continue
    
    image_annotations = [annotation for annotation in data['annotations'] if annotation['image_id'] == image_id]
    
    # Get image dimensions
    image_width = image_data['width']
    image_height = image_data['height']
    
    # Convert COCO annotations to YOLO format
    yolo_annotations = coco_to_yolo(image_annotations, image_width, image_height)
    
    if not yolo_annotations:
        print(f"No annotations found for image {image_id}")
        continue
    
    # Format image ID with leading zeros
    image_name = f"{image_id:012d}.txt"
    
    # Writing to YOLO format
    output_file = os.path.join(output_path, image_name)
    with open(output_file, 'w') as f:
        for annotation in yolo_annotations:
            category_id, x_center, y_center, width, height = annotation
            line = f"{category_id} {x_center} {y_center} {width} {height}\n"
            f.write(line)
    
    print(f"Annotations for image {image_id} converted to YOLO format and saved to {output_file}")


Image 391895 not found, skipping...
Image 522418 not found, skipping...
Image 184613 not found, skipping...
Image 318219 not found, skipping...
Image 554625 not found, skipping...
Image 574769 not found, skipping...
Annotations for image 60623 converted to YOLO format and saved to C:\Users\12108\fiftyone\detect\labels\labels_train3\000000060623.txt
Image 309022 not found, skipping...
Annotations for image 5802 converted to YOLO format and saved to C:\Users\12108\fiftyone\detect\labels\labels_train3\000000005802.txt
Image 222564 not found, skipping...
Image 118113 not found, skipping...
Image 193271 not found, skipping...
Image 224736 not found, skipping...
Image 483108 not found, skipping...
Image 403013 not found, skipping...
Image 374628 not found, skipping...
Image 328757 not found, skipping...
Image 384213 not found, skipping...
Image 293802 not found, skipping...
Image 86408 not found, skipping...
Image 372938 not found, skipping...
Image 386164 not found, skipping...
Image 223648

In [15]:
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])